In [43]:
import sqlite3
import pandas as pd
import os
from groq import Groq

In [44]:
os.environ["GROQ_API_KEY"] = "YOUR_API_KEY"

In [59]:
conn = sqlite3.connect("school.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
id INTEGER PRIMARY KEY,
name TEXT,
age INTEGER,
marks INTEGER,
department TEXT
)
""")
conn.commit()
print("✅ Table created")

✅ Table created


In [60]:
# Clear table before inserting (prevents duplicates)
cursor.execute("DELETE FROM students")
conn.commit()

students_data = [
    (1,"Rahul", 20, 85, "CSE"),
    (2,"Anita", 21, 92, "ECE"),
    (3,"Aman", 19, 78, "ME"),
    (4,"Sneha", 22, 88, "CSE"),
    (5,"Rohit", 20, 65, "CE")
]

cursor.executemany(
    "INSERT INTO students (id, name, age, marks, department) VALUES (?, ?, ?, ?, ?)",
    students_data
)

conn.commit()
print("✅ Fresh data inserted (no duplicates)")


✅ Fresh data inserted (no duplicates)


In [63]:
pd.read_sql("SELECT * FROM students", conn)

,id,name,age,marks,department
0,1,Rahul,20,85,CSE
1,2,Anita,21,92,ECE
2,3,Aman,19,78,ME
3,4,Sneha,22,88,CSE
4,5,Rohit,20,65,CE


In [48]:
# Create departments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_code TEXT PRIMARY KEY,
    dept_name TEXT,
    hod TEXT,
    building TEXT
)
""")

conn.commit()
print("✅ Departments table created")

# Clear table before inserting
cursor.execute("DELETE FROM departments")
conn.commit()

# Insert department data
departments_data = [
    ("CSE", "Computer Science Engineering", "Dr. Sharma", "Block A"),
    ("ECE", "Electronics & Communication", "Dr. Verma", "Block B"),
    ("ME", "Mechanical Engineering", "Dr. Singh", "Block C"),
    ("CE", "Civil Engineering", "Dr. Gupta", "Block D")
]

cursor.executemany(
    "INSERT INTO departments (dept_code, dept_name, hod, building) VALUES (?, ?, ?, ?)",
    departments_data
)

conn.commit()
print("✅ Departments data inserted")


✅ Departments table created
✅ Departments data inserted


In [49]:
pd.read_sql("SELECT * FROM departments", conn)

,dept_code,dept_name,hod,building
0,CSE,Computer Science Engineering,Dr. Sharma,Block A
1,ECE,Electronics & Communication,Dr. Verma,Block B
2,ME,Mechanical Engineering,Dr. Singh,Block C
3,CE,Civil Engineering,Dr. Gupta,Block D


In [50]:
SCHEMA = """
Table: students
Columns:
- id (INTEGER, PRIMARY KEY)
- name (TEXT)
- age (INTEGER)
- marks (INTEGER)
- department (TEXT, foreign key -> departments.dept_code)

Table: departments
Columns:
- dept_code (TEXT, PRIMARY KEY)
- dept_name (TEXT)
- hod (TEXT)
- building (TEXT)

Relationship:
- students.department = departments.dept_code
"""


In [72]:
SYSTEM_PROMPT = f"""
You are an expert NL2SQL assistant. And your task is to create SQL lite queries from the database schema.

STRICT RULES:
1. Use ONLY provided tables and columns
2. SQLite syntax only
3. NO explanations
4. NO comments
5. Only SELECT queries
6. If question is out of context or beyond schema, mention like I'm designed to provide SQL Queries for the Database


Schema:
{SCHEMA}
"""

In [73]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [74]:
def generate_sql(question):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
        ],
        temperature=0
    )
    return response.choices[0].message.content.strip()

In [78]:
def run_query(sql):
    try:
        conn = sqlite3.connect("school.db")
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return pd.DataFrame(rows, columns=columns)
    except Exception as e:
        return None

In [119]:
question = "2nd highest Marks with detail"

sql_query = generate_sql(question)
print("Generated SQL:\n", sql_query)

result_df = run_query(sql_query)
result_df

Generated SQL:
 SELECT name, age, marks, department FROM students WHERE marks = ( SELECT DISTINCT marks FROM students ORDER BY marks DESC LIMIT 1 OFFSET 1 )


,name,age,marks,department
0,Sneha,22,88,CSE
